In [16]:
import pandas as pd
import numpy as np
import requests
import json
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.endpoints import teamdashlineups
from nba_api.stats.static import teams
from nba_api.stats.endpoints import teamdashboardbygeneralsplits
from nba_api.stats.endpoints import leaguedashteamstats
from nba_api.stats.static import players
from nba_api.stats.endpoints import shotchartdetail
from nba_api.stats.endpoints import playercareerstats
from scipy.stats import norm, gaussian_kde, percentileofscore
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import leaguedashteamstats
pd.set_option('display.max_columns',None)
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
from matplotlib.patches import Circle, Rectangle, Arc, ConnectionPatch
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from matplotlib.colors import LinearSegmentedColormap, ListedColormap, BoundaryNorm
from matplotlib.path import Path
from matplotlib.patches import PathPatch
import os
sns.set_style('white')
sns.set_color_codes()


In [12]:
def win(team, ssn):
    temp = teams.find_team_by_abbreviation(str(team))
    team_id = temp['id']
    team_name = temp['full_name']
    
    data = leaguedashteamstats.LeagueDashTeamStats(season=ssn)
    df = data.get_data_frames()[0]
    df = df[df['TEAM_ID'] == team_id]
    df = df[['W_RANK']]
    return df
#win('BOS','2019-20')

,W_RANK
1,5


In [17]:
def plusminus(tm, ssn):

    headers = {'Host': 'stats.nba.com','Accept': 'application/json, text/plain, /','x-nba-stats-token': 'true','User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Mobile Safari/537.36','x-nba-stats-origin': 'stats','Origin': 'https://www.nba.com','Referer': 'https://www.nba.com/','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9}'}
    
    temp = teams.find_team_by_abbreviation(str(tm))
    team_id = temp['id']
    
    
    url = 'https://stats.nba.com/stats/teamdashlineups?DateFrom=&DateTo=&GameID=&GameSegment=&GroupQuantity=5&LastNGames=0&LeagueID=&Location=&MeasureType=Base&Month=0&OpponentTeamID=0&Outcome=&PORound=&PaceAdjust=N&PerMode=Totals&Period=0&PlusMinus=N&Rank=N&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&ShotClockRange=&TeamID=' + str(team_id) + '&VsConference=&VsDivision='
    
    
    json = requests.get(url, headers=headers).json()

    data = json['resultSets'][1]['rowSet']
    columns = json['resultSets'][1]['headers']
    df = pd.DataFrame.from_records(data, columns=columns)

    df = df[['GROUP_NAME','MIN','PLUS_MINUS','W_PCT']]
    df = df.iloc[:10]

    fig = px.scatter(df, x='PLUS_MINUS', y='W_PCT', size='MIN',custom_data='GROUP_NAME',labels={"size"})

    fig.update_traces(
    hovertemplate='LINEUP: %{customdata[0]}<br>PLUS MINUS: %{x}<br>WIN%: %{y}<br>MIN: %{marker.size}'
    )
    fig.update_xaxes(zeroline=True, zerolinecolor='dodgerblue', zerolinewidth=1)
    fig.update_yaxes(zeroline=True, zerolinecolor='dodgerblue', zerolinewidth=1)

    return fig
plusminus('BOS','2019-20')


In [18]:
def radarplot(team, ssn):
    temp = teams.find_team_by_abbreviation(str(team))
    team_id = temp['id']
    team_name = temp['full_name']

    data  = leaguedashteamstats.LeagueDashTeamStats(season = ssn)
    df = data.get_data_frames()[0]
    df = df[df['TEAM_ID'] == team_id]
    df = df[['FG_PCT_RANK','REB_RANK','AST_RANK','TOV_RANK','PLUS_MINUS_RANK']]
    columns_to_subtract = ['FG_PCT_RANK', 'REB_RANK', 'AST_RANK','PLUS_MINUS_RANK']
    df[columns_to_subtract] = df[columns_to_subtract].apply(lambda x: 31 - x)
    
    df.columns = ['Field Goal%','Rebound','Assist','Turnover','Plus Minus']

    fig = px.line_polar(r=[df['Field Goal%'].values[0],df['Rebound'].values[0],df['Assist'].values[0],df['Turnover'].values[0],df['Plus Minus'].values[0]],
                    theta=['Field Goal%', 'Rebound', 'Assist', 'Turnover', 'Plus Minus'],
                    line_close=True, title=team_name)
    
    fig.update_traces(fill='toself')
    return fig
radarplot('BOS','2019-20')


In [20]:
def monthbymonth(team,ssn):
    temp = teams.find_team_by_abbreviation(str(team))
    team_id = temp['id']
    team_name = temp['full_name']

    data = teamdashboardbygeneralsplits.TeamDashboardByGeneralSplits(team_id=team_id,season=ssn)
    df = data.get_data_frames()[3]

    df = df[['GROUP_VALUE','W_PCT']]
    df.columns = ['MONTH','W%']

    fig = px.line(df, x="MONTH", y="W%", markers='True')
    
    return fig
monthbymonth('BOS','2019-20')


In [7]:
def get_player_shotchartdetail(team, ssn):
    temp = teams.find_team_by_abbreviation(str(team))
    team_id = temp['id']
    team_name = temp['full_name']
    
    temp = leaguedashplayerstats.LeagueDashPlayerStats(team_id_nullable=team_id, season=ssn)
    players = temp.get_data_frames()[0]
    players = players[players.MIN>200]
    players = players.PLAYER_ID.unique()
    df_list = []
    for x in players:
        shotchartlist = shotchartdetail.ShotChartDetail(team_id = team_id, season_nullable= ssn, player_id=x,context_measure_simple="FGA")
        data = shotchartlist.get_data_frames()[0]
        
        df_list.append(data)

    df = pd.concat(df_list)
    avg = shotchartlist.get_data_frames()[1]
    return df, avg


In [8]:
def draw_court(ax=None, color="blue", lw=1, shotzone=False, outer_lines=False):
   
    if ax is None:
        ax = plt.gca()

   
    hoop = Circle((0, 0), radius=7.5, linewidth=lw, color=color, fill=False)

    backboard = Rectangle((-30, -12.5), 60, 0, linewidth=lw, color=color)

    outer_box = Rectangle((-80, -47.5), 160, 190, linewidth=lw, color=color,
                          fill=False)

    inner_box = Rectangle((-60, -47.5), 120, 190, linewidth=lw, color=color,
                          fill=False)

    top_free_throw = Arc((0, 142.5), 120, 120, theta1=0, theta2=180,
                         linewidth=lw, color=color, fill=False)

    bottom_free_throw = Arc((0, 142.5), 120, 120, theta1=180, theta2=0,
                            linewidth=lw, color=color, linestyle='dashed')
    
    restricted = Arc((0, 0), 80, 80, theta1=0, theta2=180, linewidth=lw,
                     color=color)

    corner_three_a = Rectangle((-220, -47.5), 0, 140, linewidth=lw,
                               color=color)

    corner_three_b = Rectangle((220, -47.5), 0, 140, linewidth=lw, color=color)

    three_arc = Arc((0, 0), 475, 475, theta1=22, theta2=158, linewidth=lw,
                    color=color)

 
    center_outer_arc = Arc((0, 422.5), 120, 120, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    center_inner_arc = Arc((0, 422.5), 40, 40, theta1=180, theta2=0,
                           linewidth=lw, color=color)
    
    if (shotzone == True):
        inner_circle = Circle((0, 0), radius=80, linewidth=lw, color='black', fill=False)
        outer_circle = Circle((0, 0), radius=160, linewidth=lw, color='black', fill=False)
        corner_three_a_x =  Rectangle((-250, 92.5), 30, 0, linewidth=lw, color=color)
        corner_three_b_x = Rectangle((220, 92.5), 30, 0, linewidth=lw, color=color)
        
        inner_line_1 = Rectangle((40, 69.28), 80, 0, 60, linewidth=lw, color=color)
       
        inner_line_2 = Rectangle((-40, 69.28), 80, 0, 120, linewidth=lw, color=color)
        
        inner_line_3 = Rectangle((53.20, 150.89), 290, 0, 70.53, linewidth=lw, color=color)
        inner_line_4 = Rectangle((-53.20, 150.89), 290, 0, 109.47, linewidth=lw, color=color)

        inner_line_5 = Rectangle((130.54, 92.5), 80, 0, 35.32, linewidth=lw, color=color)
        inner_line_6 = Rectangle((-130.54, 92.5), 80, 0, 144.68, linewidth=lw, color=color)
        
       
        court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                          bottom_free_throw, restricted, corner_three_a,
                          corner_three_b, three_arc, center_outer_arc,
                          center_inner_arc, inner_circle, outer_circle,
                          corner_three_a_x, corner_three_b_x,
                          inner_line_1, inner_line_2, inner_line_3, inner_line_4, inner_line_5, inner_line_6]
    else:
        court_elements = [hoop, backboard, outer_box, inner_box, top_free_throw,
                          bottom_free_throw, restricted, corner_three_a,
                          corner_three_b, three_arc, center_outer_arc,
                          center_inner_arc]
    
    if outer_lines:
        outer_lines = Rectangle((-250, -47.5), 500, 470, linewidth=lw,
                                color=color, fill=False)
        court_elements.append(outer_lines)

    for element in court_elements:
        ax.add_patch(element)
        

    return ax

In [9]:
def sized_hexbin(ax, hc, hc2, cmap, norm):
    offsets = hc.get_offsets()
    orgpath = hc.get_paths()[0]
    verts = orgpath.vertices
    values1 = hc.get_array()
    values2 = hc2.get_array()
    ma = values1.max()
    patches = []

    for offset,val in zip(offsets,values1):
  
        filtered_list = list(filter(lambda num: num != 0, values1))

        if (int(val) == 0):
            continue
        elif (percentileofscore(filtered_list, val) < 33.33):

            v1 = verts*0.3 + offset
        elif (percentileofscore(filtered_list, val) > 69.99):

            v1 = verts + offset
        else:

            v1 = verts*0.6 + offset
        
        path = Path(v1, orgpath.codes)
        patch = PathPatch(path)
        patches.append(patch)

    pc = PatchCollection(patches, cmap=cmap, norm=norm)

    pc.set_array(values2)

    ax.add_collection(pc)
    hc.remove()
    hc2.remove()

def hexmap_chart(data, league_avg, title="", color="b",
               xlim=(-250, 250), ylim=(422.5, -47.5), line_color="white",
               court_color="#1a477b", court_lw=2, outer_lines=False,
               flip_court=False, gridsize=None, 
               ax=None, despine=False, **kwargs):
    
    LA = league_avg.loc[:,['SHOT_ZONE_AREA','SHOT_ZONE_RANGE', 'FGA', 'FGM']].groupby(['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE']).sum()
    LA['FGP'] = 1.0*LA['FGM']/LA['FGA']
    player = data.groupby(['SHOT_ZONE_AREA','SHOT_ZONE_RANGE','SHOT_MADE_FLAG']).size().unstack(fill_value=0)
    player['FGP'] = 1.0*player.loc[:,1]/player.sum(axis=1)
    player_vs_league = (player.loc[:,'FGP'] - LA.loc[:,'FGP'])*100  

    data = pd.merge(data, player_vs_league, on=['SHOT_ZONE_AREA', 'SHOT_ZONE_RANGE'], how='right')
    
    if ax is None:
        ax = plt.gca()
        ax.set_facecolor(court_color)
    if not flip_court:
        ax.set_xlim(xlim)
        ax.set_ylim(ylim)
    else:
        ax.set_xlim(xlim[::-1])
        ax.set_ylim(ylim[::-1])

    ax.tick_params(labelbottom="off", labelleft="off")
    ax.set_title(title, fontsize=18)

    # draws the court
    draw_court(ax, color=line_color, lw=court_lw, outer_lines=outer_lines)

    x = data['LOC_X']
    y = data['LOC_Y']
 
    colors = ['#003366','#2b7cb6', '#abd9e9', '#ffffbf', '#fdaf61', '#d7191c']
    cmap = ListedColormap(colors)
    
    boundaries = [-np.inf, -9, -3, 0, 3, 9, np.inf]
    norm = BoundaryNorm(boundaries, cmap.N, clip=True)   
    
    hexbin = ax.hexbin(x, y, gridsize=40, cmap=cmap, norm=norm, extent=[-275, 275, -50, 425])
    hexbin2 = ax.hexbin(x, y, C=data['FGP'], gridsize=40, cmap=cmap, norm=norm, extent=[-275, 275, -50, 425])
    sized_hexbin(ax, hexbin, hexbin2, cmap, norm) 
    
    for spine in ax.spines:
        ax.spines[spine].set_lw(court_lw)
        ax.spines[spine].set_color(line_color)

    if despine:
        ax.spines["top"].set_visible(False)
        ax.spines["bottom"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.spines["left"].set_visible(False)

    return ax

In [10]:
def create_heat(team, ssn):
    player_shotchart_df, league_avg = get_player_shotchartdetail(team, ssn)
    fig = hexmap_chart(player_shotchart_df, league_avg)
    return fig
#create_heat('BOS','2018-19')


In [11]:
import matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import numpy as np
import requests
import json
from nba_api.stats.endpoints import leaguedashplayerstats
from nba_api.stats.endpoints import playerdashptpass
from nba_api.stats.static import teams

pd.set_option('display.max_columns',None)

def pass_web(team,ssn,web): 
    
    ### the 'team' parameter takes in an nba team's three letter abbreviation. 'GSW' for the golden state warriors, for example
    ### the 'ssn' parameter takes in an nba season (2013-14 at the earliest). '2015-16' for example
    ### the 'web' parameter takes in 'AST' or 'pass' (or other variations of the two). 
    ### the 'web' parameter exists because you can create networks in which the lines represent either assists or passes -- the parameter is used to tell which one of the two you'd like to do
    ### example of usage: pass_web('GSW','2016-17','AST')
    
    if (web in ['ast','AST','assist','assists','assisting','ASSIST','ASSISTS']):
        web = 'assist'
    elif (web in ['pass','passes','passing','PASS','PASSES']):
        web = 'pass'
    else:
        return print('Error: Third parameter requires "assist" or "pass" input')
    
    headers = {'Host': 'stats.nba.com','Accept': 'application/json, text/plain, /','x-nba-stats-token': 'true','User-Agent': 'Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Mobile Safari/537.36','x-nba-stats-origin': 'stats','Origin': 'https://www.nba.com','Referer': 'https://www.nba.com/','Accept-Encoding': 'gzip, deflate, br','Accept-Language': 'en-US,en;q=0.9}'}

    temp = teams.find_team_by_abbreviation(str(team))
    team_id = temp['id']
    team_name = temp['full_name']

    temp = leaguedashplayerstats.LeagueDashPlayerStats(team_id_nullable=team_id, season=ssn)
    df = temp.get_data_frames()[0]
   
    df = df[df.AST > 50]
    
    #print(df)
    players = df.PLAYER_ID.unique()
    
    ### using the players previously found, record all of their pass connections while on the team

    df_list = []
    for x in players:

        # url = 'https://stats.nba.com/stats/playerdashptpass?DateFrom=&DateTo=&GameSegment=&LastNGames=0&LeagueID=00&Location=&Month=0&OpponentTeamID=0&Outcome=&PORound=0&PerMode=Totals&Period=0&PlayerID=' + str(x) + '&Season=' + str(ssn) + '&SeasonSegment=&SeasonType=Regular+Season&TeamID=0&VsConference=&VsDivision='

        # json = requests.get(url, headers=headers).json()

        # data = json['resultSets'][0]['rowSet']
        # columns = json['resultSets'][0]['headers']
        data = playerdashptpass.PlayerDashPtPass(season=ssn, player_id=x,team_id=team_id)
        temp = data.get_data_frames()[0]
        
        df_list.append(temp)
    
    df = pd.concat(df_list)
    df = df[['TEAM_ABBREVIATION','PLAYER_NAME_LAST_FIRST','PASS_TO','PASS','AST']]
    df.columns = ['tm','passer','receiver','passes','ast']
    ### clean up the format for the name

    def fix_name(name):
        if ',' in name:
            return name.split(", ")[1][:1] + "." + name.split(", ")[0]
        else:
            return name
    df.passer = np.vectorize(fix_name)(df.passer)
    df.receiver = np.vectorize(fix_name)(df.receiver)

    df = df[df.receiver.isin(df.passer.unique())].reset_index(drop=True)

    players = df.passer.unique()
    
    ### making list with assist totals for each player (will be used as input for node size)
    
    ast_list = []
    for x in players:
        tf = df[df.passer == x].reset_index(drop=True)
        ast_list.append(tf.ast.sum())
        
    ### creating adjacency matrix

    adf = pd.DataFrame(index=players,columns=players)

    adf.values[[np.arange(adf.shape[0])]*2] = 0
    if web == 'pass':
        for x in players:
            for y in range(0,len(players)):
                tf1 = df[(df.passer == x) & (df.receiver == adf.columns[y])].reset_index(drop=True)
                tf2 = df[(df.passer == adf.columns[y]) & (df.receiver == x)].reset_index(drop=True)
                if (tf1.shape[0] == 1) & (tf2.shape[0] == 0):
                    adf.at[x,adf.columns[y]] = tf1.passes[0]
                elif (tf1.shape[0] == 0) & (tf2.shape[0] == 1):
                    adf.at[x,adf.columns[y]] = tf2.passes[0]
                elif (tf1.shape[0] == 1) & (tf2.shape[0] == 1):
                    adf.at[x,adf.columns[y]] = tf1.passes[0] + tf2.passes[0]
                else:
                    adf.at[x,adf.columns[y]] = 0
                    
    else:
        for x in players:
            for y in range(0,len(players)):
                tf1 = df[(df.passer == x) & (df.receiver == adf.columns[y])].reset_index(drop=True)
                tf2 = df[(df.passer == adf.columns[y]) & (df.receiver == x)].reset_index(drop=True)
                if (tf1.shape[0] == 1) & (tf2.shape[0] == 0):
                    adf.at[x,adf.columns[y]] = tf1.ast[0]
                elif (tf1.shape[0] == 0) & (tf2.shape[0] == 1):
                    adf.at[x,adf.columns[y]] = tf2.ast[0]
                elif (tf1.shape[0] == 1) & (tf2.shape[0] == 1):
                    adf.at[x,adf.columns[y]] = tf1.ast[0] + tf2.ast[0]
                else:
                    adf.at[x,adf.columns[y]] = 0
                    
    ### graphing the network
    
    plt.figure(figsize=(8,8))

    G = nx.from_numpy_matrix(np.array(adf.values, dtype=int), parallel_edges=True)
    
    d = dict(G.degree)
#     print(d.keys().index)
#     return

    edges,weights = zip(*nx.get_edge_attributes(G,'weight').items())

    pos = nx.circular_layout(G, center=(0,0))
    
    labeldict = {}
    for n in range(0,len(adf.index)):
        labeldict[n] = adf.index[n]

    nx.draw_networkx(G,with_labels=False, nodelist=d.keys(), node_size=[v for v in ast_list],linewidths=2, edgelist=edges, edge_color=weights, width=2, edge_cmap=plt.cm.binary, node_color='white', pos=pos)
    #nx.draw_circular(G, nodelist=list(d), node_size=[v for v in ast_list],linewidths=2, edgelist=edges, edge_color=weights, width=2, edge_cmap=plt.cm.binary, node_color='white')
    
    for p in pos.keys():
        cord = pos[p]
        offset_x, offset_y = 0.12, 0.12
        if (cord[0] < 0):
            offset_x *= -1
        if (cord[1] < 0):
            offset_y *= -1
        cord[0] += offset_x
        cord[1] += offset_y
        pos[p] = cord

    nx.draw_networkx_labels(G, pos, labels=labeldict, horizontalalignment='center')
    
    ax = plt.gca()
    ax.collections[0].set_edgecolor("#000000") 
    
    if web == 'pass':
        plt.title(ssn + ' ' + team_name + ' Passing Network')
    else:
        plt.title(ssn + ' ' + team_name + ' Assist Network')
        
    dir_name = "/Users/prashantmanandhar/Desktop/data vis/assets/"

 
    plt.savefig(dir_name+team + ssn+ '.png',dpi=300,bbox_inches='tight')


# pass_web('ATL','2017-18','AST')
# pass_web('ATL','2018-19','AST')
# pass_web('ATL','2020-21','AST')
# pass_web('LAL','2016-17','AST')
# pass_web('BOS','2017-18','AST')
# pass_web('BOS','2018-19','AST')
# pass_web('BOS','2020-21','AST')